In [ ]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig

### Helper functions

In [ ]:
def save_result(path_name, file_name, result):
    """
    Save the result to a file.
    
    Args:
        path_name (str): Path to save the file
        file_name (str): Name of the file
        result (str): Content to save
    """
    file_path = os.path.join(path_name, f"{file_name}.md")
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    with open(file_path, 'w') as file:
        file.write(result)
        
def setup_environment():
    """Set up the environment variables."""
    try:
        hf_key = os.environ["LLAMA3_KEY"]
    except KeyError:
        print("Please set the environment variable LLAMA3_KEY")
        hf_key = input("Enter your HuggingFace API key: ")
    return hf_key

def load_model(hf_key):
    """Load the LLM model and tokenizer."""
    quant_config = BitsAndBytesConfig(load_in_8bit=True)
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
    
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_key)
    model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_key, quantization_config=quant_config, device_map="auto", max_length=8192)
    config = AutoConfig.from_pretrained(model_name, token=hf_key)
    
    return model, tokenizer, config

def generate_response(model, tokenizer, messages, config):
    """Generate a response from the model."""
    model_inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to('cuda')

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    generated_ids = model.generate(
        model_inputs,
        do_sample=True,
        temperature=0.1,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        max_length=4500
    )

    print(f"Tokens used: {len(model_inputs[0])} out of {config.max_position_embeddings}")

    response = generated_ids[0][model_inputs.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

## 1. Set up the model

This step includes downloading the LLM from Hugging Face and setting up the API key.

You can temporarily add your API key by using Python with os.environ["LLAMA3_KEY"]="YOUR_API_KEY"

WARNING: NEVER SHARE/PUSH YOUR API KEY.

In [ ]:
hf_key = setup_environment()
model, tokenizer, config = load_model(hf_key)

## 2. Set file directories

### Sample files and templates for the LLM

In [ ]:
analysis_template_path = "files/templates/analysis-template.md"
sample_case_path = "files/templates/sample-case.md"
sample_case_2_path = "files/templates/sample-case-2.md"
sample_analysis_path = "files/templates/sample-analysis.md"
sample_analysis_2_path = "files/templates/sample-analysis-2.md"

analysis_template = open(analysis_template_path, "r").read()

sample_case = open(sample_case_path, "r").read()
sample_analysis = open(sample_analysis_path, "r").read()

sample_case_2 = open(sample_case_2_path, "r").read()
sample_analysis_2 = open(sample_analysis_2_path, "r").read()

### Individual case to analyze

In [ ]:
case_name = "norauto"

file_path = "files/cases/" +  case_name + "/description.md"
case_description = open(file_path, "r").read()

## 3. Start Analysis

In [ ]:
messages = [
    {"role": "system", "content": f"You are an expert in AI ethics, law, and engineering, specializing in addressing discrimination and fairness issues in models. Please conduct a detailed analysis of the provided dataset, considering ethical, legal, and engineering aspects related to fairness and bias. The analysis should be structured in a markdown format with the following sections: \n{analysis_template}"},
    {"role": "user", "content": f"Data and use case: {sample_case}"},
    {"role": "assistant", "content": sample_analysis},
    {"role": "user", "content": f"This is a valid analysis. Please do an analysis with this dataset and use case: {sample_case_2}"},
    {"role": "assistant", "content": sample_analysis_2},
    {"role": "user", "content": f"This is a valid analysis. Please do an analysis with this dataset and use case: {case_description}"},
]

In [ ]:
result = generate_response(model, tokenizer, messages, config)
print(result)

In [ ]:
case_path = "files/cases/" + case_name + "/"
save_result(case_path, "ethics-analysis", result)